In [1]:
import clingo
from json import dumps
import re

In [2]:
def combine(encoding_path, benchmark_path, destination_path):
    
    combined_path = destination_path
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [3]:
def save(solution, destination_path):
    file = open(destination_path, "w")
    file.write(solution)
    file.close()

In [4]:
def save_plans(plans, destination_path):
    file = open(destination_path, "w")
    for i in range(len(plans)):
        file.write(plans[i])
    file.close()

In [5]:
def prep(destination_path, time):
    combined_path = destination_path
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    solution = ""
    md=[]
    gr=[]
    asp=combined

    ctl = clingo.Control(['--stats'])
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: 
            solution=("{} ".format(m))
            md=re.findall('md(.+?) ', ("{} ".format(m)))
            gr=re.findall('group(.+?) ', ("{} ".format(m)))
        handle.get()
        solution = solution.replace(' ', '. ')
    time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
    print(dumps(ctl.statistics['summary']['times'], sort_keys=True, indent=4, separators=(',', ': ')))
    return solution, gr, md, time

In [6]:
def plan(plan, groups, man_dis, destination_path, time):
    combined_path = destination_path
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    

    for i in groups:
        solution = ""
        asp_= ""
        grp = "#const grp = {}.".format(i)
        asp = combined + grp
        for j in range(man_dis,man_dis*2):
            horizon = "#const horizon = {}.".format(j)
            asp = asp + horizon
            ctl = clingo.Control(['--stats'])
            ctl.add("base", [], asp)
            ctl.ground([("base", [])])
            with ctl.solve(yield_=True) as handle:
                for m in handle: 
                    solution = ("{} ".format(m))
                handle.get()
                solution = solution.replace(' ', '. ')
            if solution:
                time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
                plans.append(solution)
                break
    return plans, time

In [7]:
enc1 = "./asprilo-encodings/group/path.lp"
enc2 = "./asprilo-encodings/group/prep.lp"
enc3 = "./asprilo-encodings/group/prep_all.lp"
enc4 = "./asprilo-encodings/group/prep_all_og.lp"
abst = "./instances/adrian/benchmark2/CNC/cnc_instance.lp"
inst = "./instances/adrian/benchmark2/instance.lp"
dest1 = "./solution/combined_instance.lp"
dest2 = "./solution/combined_prep.lp"
dest3 = "./solution/prep.lp"
dest4 = "./solution/plan.lp"


In [8]:
time=[]
plans=[]
combine(enc2,abst,dest2)

prep_, groups, man_dis, time = prep(dest2, time)
save(prep_,dest3)
combine(enc1,dest3,dest1)

for i in range(len(groups)):
    groups[i] = int(groups[i].strip('()'))
md=[]
for i in range(len(man_dis)):
    x,y = map(int, man_dis[i].strip('()').split(','))
    md.append(y)
max_md = max(md)
print(md)

plans, time = plan(plans, groups, max_md, dest1, time)
save_plans(plans, dest4)
print(sum(time))

{
    "cpu": 0.03237200000000007,
    "sat": 6.699562072753906e-05,
    "solve": 0.005391120910644531,
    "total": 0.031965017318725586,
    "unsat": 0.0
}
[3, 13, 7, 5, 1, 15, 9, 5, 4, 6, 15, 0, 2, 9, 10, 2, 5, 22, 9, 19]
1.180859088897705


In [9]:
"""
time_all=[]
plans_all=[]
combine(enc3,abst,dest2_all)

prep_all, groups_all, man_dis_all, time_all = prep(dest2_all, time_all)
save(prep_all,dest3_all)
combine(enc1,dest3_all,dest1_all)

for i in range(len(groups_all)):
    groups[i] = int(groups_all[i].strip('()'))
md_all=[]
for i in range(len(man_dis_all)):
    x,y = map(int, man_dis_all[i].strip('()').split(','))
    md_all.append(y)
max_md_all = max(md_all)

plans_all, time_all = plan(plans_all, groups_all, max_md_all, dest1_all, time_all)
save_plans(plans_all, dest4_all)
print(sum(time_all))
"""

"\ntime_all=[]\nplans_all=[]\ncombine(enc3,abst,dest2_all)\n\nprep_all, groups_all, man_dis_all, time_all = prep(dest2_all, time_all)\nsave(prep_all,dest3_all)\ncombine(enc1,dest3_all,dest1_all)\n\nfor i in range(len(groups_all)):\n    groups[i] = int(groups_all[i].strip('()'))\nmd_all=[]\nfor i in range(len(man_dis_all)):\n    x,y = map(int, man_dis_all[i].strip('()').split(','))\n    md_all.append(y)\nmax_md_all = max(md_all)\n\nplans_all, time_all = plan(plans_all, groups_all, max_md_all, dest1_all, time_all)\nsave_plans(plans_all, dest4_all)\nprint(sum(time_all))\n"

In [10]:
"""
time_all_og=[]
plans_all_og=[]
combine(enc4,inst,dest2_all_og)

prep_all_og, groups_all_og, man_dis_all_og, time_all_og = prep(dest2_all_og, time_all_og)
save(prep_all_og,dest3_all_og)
combine(enc1,dest3_all_og,dest1_all_og)

for i in range(len(groups_all_og)):
    groups[i] = int(groups_all_og[i].strip('()'))
md_all_og=[]
for i in range(len(man_dis_all_og)):
    x,y = map(int, man_dis_all_og[i].strip('()').split(','))
    md_all_og.append(y)
max_md_all_og = max(md_all_og)

plans_all_og, time_all_og = plan(plans_all_og, groups_all_og, max_md_all_og, dest1_all_og, time_all_og)
save_plans(plans_all_og, dest4_all_og)
print(sum(time_all_og))
"""

"\ntime_all_og=[]\nplans_all_og=[]\ncombine(enc4,inst,dest2_all_og)\n\nprep_all_og, groups_all_og, man_dis_all_og, time_all_og = prep(dest2_all_og, time_all_og)\nsave(prep_all_og,dest3_all_og)\ncombine(enc1,dest3_all_og,dest1_all_og)\n\nfor i in range(len(groups_all_og)):\n    groups[i] = int(groups_all_og[i].strip('()'))\nmd_all_og=[]\nfor i in range(len(man_dis_all_og)):\n    x,y = map(int, man_dis_all_og[i].strip('()').split(','))\n    md_all_og.append(y)\nmax_md_all_og = max(md_all_og)\n\nplans_all_og, time_all_og = plan(plans_all_og, groups_all_og, max_md_all_og, dest1_all_og, time_all_og)\nsave_plans(plans_all_og, dest4_all_og)\nprint(sum(time_all_og))\n"